# ]-----------------------------[Titanic Survival Prediction]-----------------------------------[

In [1]:
import numpy as np
import pandas as pd
import polars as pl

import sklearn
from sklearn.model_selection import train_test_split



In [2]:
#Dataset
data=pd.read_csv('dataset_vs/tested.csv')
data2=pl.read_csv('dataset_vs/tested.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [4]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
# Defining Target and deleting least relevent columns
X=data.copy()
X=X.sample(len(X))
y=X['Survived']
X=X.drop(columns=["Name","Survived","Cabin","Ticket"])
X.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
19,911,3,female,45.0,0,0,7.2250,C
132,1024,3,female,NaN,0,4,25.4667,S
52,944,2,female,20.0,2,1,23.0000,S
190,1082,2,male,34.0,1,0,26.0000,S
187,1079,3,male,17.0,2,0,8.0500,S


In [6]:
# Handling categorical values
cat_columns=['Sex','Embarked']
sex_col=X[["Sex"]]
emb_col=X[["Embarked"]]

from sklearn.preprocessing import OneHotEncoder
oh_encoder=OneHotEncoder(handle_unknown='ignore',sparse=False)
oh_col=pd.DataFrame(oh_encoder.fit_transform(X[["Sex","Embarked"]]))
oh_col.index=X.index
X=X.drop(columns=["Sex","Embarked"])

X=pd.concat([X,oh_col],axis=1)
X.columns = X.columns.astype(str)


/home/fansan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
#cat_columns=[col for col in X.columns if X.dtypes==object]
X

,PassengerId,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4
19,911,3,45.0,0,0,7.2250,1.0,0.0,1.0,0.0,0.0
132,1024,3,NaN,0,4,25.4667,1.0,0.0,0.0,0.0,1.0
52,944,2,20.0,2,1,23.0000,1.0,0.0,0.0,0.0,1.0
190,1082,2,34.0,1,0,26.0000,0.0,1.0,0.0,0.0,1.0
187,1079,3,17.0,2,0,8.0500,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
157,1049,3,23.0,0,0,7.8542,1.0,0.0,0.0,0.0,1.0
37,929,3,21.0,0,0,8.6625,1.0,0.0,0.0,0.0,1.0
180,1072,2,30.0,0,0,13.0000,0.0,1.0,0.0,0.0,1.0
409,1301,3,3.0,1,1,13.7750,1.0,0.0,0.0,0.0,1.0


In [8]:
# handling missing values
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy="median")
X_imputed=pd.DataFrame(imputer.fit_transform(X))
X_imputed.columns=X.columns
X=X_imputed

In [9]:
X

,PassengerId,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4
0,911.0,3.0,45.0,0.0,0.0,7.2250,1.0,0.0,1.0,0.0,0.0
1,1024.0,3.0,27.0,0.0,4.0,25.4667,1.0,0.0,0.0,0.0,1.0
2,944.0,2.0,20.0,2.0,1.0,23.0000,1.0,0.0,0.0,0.0,1.0
3,1082.0,2.0,34.0,1.0,0.0,26.0000,0.0,1.0,0.0,0.0,1.0
4,1079.0,3.0,17.0,2.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
413,1049.0,3.0,23.0,0.0,0.0,7.8542,1.0,0.0,0.0,0.0,1.0
414,929.0,3.0,21.0,0.0,0.0,8.6625,1.0,0.0,0.0,0.0,1.0
415,1072.0,2.0,30.0,0.0,0.0,13.0000,0.0,1.0,0.0,0.0,1.0
416,1301.0,3.0,3.0,1.0,1.0,13.7750,1.0,0.0,0.0,0.0,1.0


In [10]:
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=1)

In [11]:
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
# Linear
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier

# Tree
from sklearn.tree import DecisionTreeClassifier

# Emsembles
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier


In [12]:
model_sgdc=SGDClassifier(random_state=1)
model_pac=PassiveAggressiveClassifier(random_state=1)
model_dtc=DecisionTreeClassifier(random_state=1)
model_rfc=RandomForestClassifier(random_state=1,n_estimators=100)
model_gbc=GradientBoostingClassifier(random_state=1,n_estimators=100)
model_abc=AdaBoostClassifier(random_state=1,n_estimators=100)

models=[model_sgdc,model_gbc,model_abc,model_dtc,model_rfc,model_pac]

for i in models:
    i.fit(X_train,y_train)
    pred=i.predict(X_valid)
    score=mean_absolute_error(y_valid,pred)
    print(f"{i} MAE = ",score)    

SGDClassifier(random_state=1) MAE =  0.6071428571428571
GradientBoostingClassifier(random_state=1) MAE =  0.0
AdaBoostClassifier(n_estimators=100, random_state=1) MAE =  0.0
DecisionTreeClassifier(random_state=1) MAE =  0.0
RandomForestClassifier(random_state=1) MAE =  0.0
PassiveAggressiveClassifier(random_state=1) MAE =  0.6071428571428571


266

266